In [59]:
import pandas as pd
from datetime import date, datetime, timedelta
import json
import time
from riotwatcher import LolWatcher, ApiError

## Get the summoners' Ids

In [110]:
from riotwatcher import LolWatcher, ApiError
# api_key = 'RGAPI-53949b86-ecf1-4867-9e1b-5f4186324a73'
api_key = 'RGAPI-4ac683d9-a5e9-4f8b-9c38-d22993d16e57'
lol_watcher = LolWatcher(api_key)

my_region = 'na1'
me = lol_watcher.summoner.by_name(my_region, 'mengbier')
print(me)

{'id': 'Y19I_4m4qTj0Kb0ZXiuI_OLIELfEt-VxLhnfUbKjI92dMCn1CuUGw88Q3g', 'accountId': 'EJJit2ULiT9AwwUqnADMZwwrQKFWvkZ52hYhqU5VFmvsqBYjG4Enmmzo', 'puuid': 'KSMdr1k6F2j7ic8wJfQg-KQCMT6mysKUNfa8PS8ghd0MgJdCQuinlySna8DnHrPnsUns8Rlch0XLUg', 'name': 'mengbier', 'profileIconId': 3838, 'revisionDate': 1678598077912, 'summonerLevel': 293}


In [112]:
# lol_watcher.league.challenger_by_queue(my_region, 'RANKED_SOLO_5x5')['entries']

In [90]:
platforms = ["br1","eun1","euw1","jp1","kr","la1","la2","na1",
             "oc1","tr1","ru","ph2","sg2","th2","tw2","vn2"]
tiers = ['challenger', 'grandmaster', 'master']
position_map = {
    ('MID', 'SOLO'): 'middle',
    ('TOP', 'SOLO'): 'top',
    ('JUNGLE', None): 'jungle',
    ('BOT', 'DUO_CARRY'): 'bottom',
    ('BOT', 'DUO_SUPPORT'): 'utility'
}
queue_id = 420 # Ranked_solo_5x5

schema = {
    'summoners': ['platform', 'tier', 'summonerId', 'summonerName'],
    'participant_keys': [
        'puuid', 'lane', 'role', 'kills', 'deaths', 'assists', 'win', 
        'championName', 'championId', 'firstBloodKill', 'firstTowerKill', 
        'item0', 'totalDamageDealt', 'totalDamageDealtToChampions',
        'totalMinionsKilled', 'visionScore', 'goldEarned'
    ],
    'matches_player': [
        'matchId', 'platform',
        'puuid', 'lane', 'role', 'kills', 'deaths', 'assists', 'win', 
        'championName', 'championId', 'firstBloodKill', 'firstTowerKill', 
        'item0', 'totalDamageDealt', 'totalDamageDealtToChampions',
        'totalMinionsKilled', 'visionScore', 'goldEarned'
    ],
    'matches': [
        'matchId', 'platform', 'gameStartTimestamp', 'gameEndTimestamp'
        'blueBans', 'blueBaronKills', 'blueChampKills', 'blueDragKills', 
        'blueInhibKills', 'blueHeraldKills', 'blueTowerKills', 'blueWin',
        'redBans', 'redBaronKills', 'redChampKills', 'redDragKills', 
        'redInhibKills', 'redHeraldKills', 'redTowerKills', 'redWin',
    ]
}

In [29]:
end_date = datetime(2023, 3, 12)
start_date = end_date - timedelta(weeks=1)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())
start_timestamp, end_timestamp

(1678003200, 1678608000)

In [4]:
def get_summoners_list(region='na1', tier='challenger', queue='RANKED_SOLO_5x5'):
    sid_list, sname_list = [], []
    entries = None
    if tier == 'challenger':
        entries = lol_watcher.league.challenger_by_queue(region, queue)['entries']
    elif tier == 'master': 
        entries = lol_watcher.league.masters_by_queue(region, queue)['entries']
    elif tier == 'grandmaster':
        entries = lol_watcher.league.grandmaster_by_queue(region, queue)['entries']
    else:
        print('tier not supported')
        return
        
    for p in entries:
        sid_list.append(p['summonerId'])
        sname_list.append(p['summonerName'])
    return sid_list, sname_list

def make_summoners_csv():
    dfs = []
    for platform in platforms:
        for tier in tiers:
            sid_list, sname_list = get_summoners_list(region=platform, tier=tier)
            df = pd.DataFrame(
                columns=schema['summoners'],
                data={
                    'platform': [platform for i in range(len(sid_list))],
                    'tier': [tier for i in range(len(sid_list))],
                    'summonerId': sid_list,
                    'summonerName': sname_list
                }
            )
            dfs.append(df)
    summoners = pd.concat(dfs)
    summoners.to_csv('summoners.csv', index=False)

In [8]:
summoners = pd.read_csv('summoners.csv')
summoners.head()

,platform,tier,summonerId,summonerName
0,br1,challenger,nBtvfvSm-LAl6HvfN7yKSJGqVNzy_qfL7eBmqFWD-ICvrg,batman vermelho
1,br1,challenger,YRmiyU2YI2PK6xXdc4TlfUdypLlO4ScAMcSrw-iaRuwNzK...,oiiii
2,br1,challenger,l-QhwVfM7V9VtMV82y9f-RRnZlQ24CszAEWB-Ogi0mGL5w,Sleepy
3,br1,challenger,1tdg0nbHFRIErMoK77jaRnnSqIW2rYWYlHhB1Ggdbe5KRl...,sim sim
4,br1,challenger,uLzlOvUmSdfnoIXaI4zDv-C9IvHqyKpjhhq5ozpSs14U_Z...,Romântico


In [10]:
# summoners_small = summoners[(summoners.platform == 'na1') & (summoners.tier == 'challenger')].iloc[:20]

,platform,tier,summonerId,summonerName
34933,na1,challenger,jCV87mRWDFKF1gIXSSzg7hlzOWFfouILsKdTNVPn_hVqa3sf,Garrett Meyerson
34934,na1,challenger,-lE6a5v83AxS8Uup2v9zOiWHZmHt4CLalEr3KGEToYVM_SA,112U
34935,na1,challenger,XxK1QSHQq_HgzAZmoSlOe7z9ff_SxrmR5N2d0wr_-pL7Mn4,Smoothie
34936,na1,challenger,W5tKS_CmhqI8f5-emruIdVGlggpwFU4yNMT-c_c99k5FKVw,YoojungHukiriLee
34937,na1,challenger,kzpbdTzOF-zMEl2aYJ5pSucoBudv2VLU1S3TS9OKUie4eFs,Test ADC 10
34938,na1,challenger,XOaHGlEDj2bgNvCftimbDTKtijCuqwFGYMOxM_4Mk-JPZlA,lightrocket2
34939,na1,challenger,Lb3taJwDKSFCAW5nFTuwE09keygeukwsvYjNIKu0ju6pZoCz,Firn
34940,na1,challenger,zHdhPVjQ-EQZeBITQPgjaEwd4K1m5epJ4WIMI5wNPEwPBJ...,enryu
34941,na1,challenger,_06OkwnQaWBwXQweY0JMGq4GPTnwKqCSJbqnAIVqQWMyu_8,LastDanceDF
34942,na1,challenger,_BHYluj1SxoYnbCZX0AVvrGAsmYQi8ccaNp07s985f6N6C...,love Yue Ying


In [48]:
challengers = summoners[summoners.tier == 'challenger']
challengers.shape

(2659, 4)

In [54]:
challengers_by_platform = challengers.groupby('platform') \
    .apply(lambda x: iter(x.summonerId.tolist())).to_dict()
challengers_by_platform

{'br1': <list_iterator at 0x17554b3d0>,
 'eun1': <list_iterator at 0x17554bb50>,
 'euw1': <list_iterator at 0x17554bd90>,
 'jp1': <list_iterator at 0x17554bdf0>,
 'kr': <list_iterator at 0x17554b970>,
 'la1': <list_iterator at 0x17554be20>,
 'la2': <list_iterator at 0x17554ba90>,
 'na1': <list_iterator at 0x17554bfa0>,
 'oc1': <list_iterator at 0x17554bf70>,
 'ph2': <list_iterator at 0x17554b1c0>,
 'ru': <list_iterator at 0x17554b160>,
 'sg2': <list_iterator at 0x17554b250>,
 'th2': <list_iterator at 0x17554b0d0>,
 'tr1': <list_iterator at 0x17554bee0>,
 'tw2': <list_iterator at 0x17554be80>,
 'vn2': <list_iterator at 0x17554be50>}

In [64]:
challengers_by_platform = challengers.groupby('platform') \
    .apply(lambda x: iter(x.summonerId.tolist())).to_dict()
challengers_by_platform

buffer_id = 1
buffer = []

done = False
while not done:
    done = True
    for platform, it in challengers_by_platform.items():
        tmpId = next(it, None)
        if tmpId is None: continue
        else:
            done = False
            resp = None
            while resp is None:
                try:
                    resp = lol_watcher.summoner.by_id(platform, tmpId)
                except ApiError as err:
                    if err.response.status_code == 429:
                        print('Retry:', tmpId)
                        time.sleep(.5)
                    elif err.response.status_code == 404:
                        print('Acc not found:', tmpId)
                        break
                    else: raise
                    
            if resp is not None:
                buffer.append(resp)
    
    if len(buffer) >= 100:
        print(f"Dumping Buffer: id: {buffer_id}, size: {len(buffer)}")
        with open(f'challengers_{buffer_id}.json', 'w') as f:
            f.write(json.dumps(buffer))
        buffer = []
        buffer_id += 1
        
        break
        

Dumping Buffer: id: 1, size: 112


In [70]:
from pathlib import Path
cwd = Path('.')

summoners = pd.read_csv('summoners.csv')
challengers = []
grandmasters = []

for f in cwd.glob('challengers_*.json'):
    with f.open() as file:
        challengers.append(pd.read_json(file))
challengers = pd.concat(challengers)
challengers


,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,Zevcpf0uhNiq9MYWFvzGfSwN2SZapLGHjMtNlKp5TKh8vQ,TVqlqDlJ-1p2QB6CI5r-nhbPQFDKVLSKN1z2WW15dHiQyLc,XEHB-3McbZrmh5w9kiJEnJInj4S9ZYoBE7otWnEWLGF7lT...,ZEN sant,29,1678672254680,669
1,QCmhAy1pBCqLqtLCUtg6rYqcvshG1TuKjJA1AL-B7qwtIS...,G4V2OQrIMwcEvfWkTjcDYAZ3TAYgrrBfVNgGCyisrr4t1N...,4mWxCKeKZYrwvfscqTbN01B4jqyMfris10K6u-OLqcQL7n...,The Speed Freak,1636,1678659552935,180
2,luJqocI9pnHMpK3X_3itk4lqqpR1DO8ZkFOjzT_5bPg8Mdk,-cJNr1p8xPPEJ5XpX62VxQxuH4AtyalAP_UdSTG4tYwPbg,m9oKf3eiNRXwiVTv40AFzTEdWtM1NWeeEWJp3xDqxriSZB...,Excel lent mid,1646,1678466567000,363
3,JzFeGEUA3diVA-PTbXm9aqbT2vmKmPvhucBqbet80MgUKh0,m77V9wFu-IEoPh2y66GV0XDONW_9IdtnFJUBtAmQyyEEmP...,6XGMMKlH66Ga2efnX1V1hw-mJLClk1Y0OKB8VQQJQIowq7...,김빵덕1,4568,1678172316000,294
4,LggxQMXRmsBrg2tdIpTchad68Xoou2vMdzlnP3DN8b2f3mo,XCSTFOiC9hQ0ZmXGhX5uvm70J8ecUuWjql1XKNxxzWUV7z4,KVqZSLM39LavL0EgQFXjjL8WXcracx6uDTpaidVVNyj78v...,Sword Skill,3838,1678422141000,251
...,...,...,...,...,...,...,...
507,9mCsIinB9vI4PTJJoN_oXpfc5IcfDOWI4af1W4o3IpXH6U...,bq50GHJaDliuOXxnQDdHsBdqw8ihtfpa-MMKyahDA_29nt...,hnktI1WcrT5zAFTtV3CWFnTJnHHZIov9Z6lBr2UM1BYwlu...,Besha,916,1678613367363,676
508,jQs1JMftfaT_CxKldD1uMLMpgrSyGpX7hsatJxhhIhPZV1...,WwUQJXcK0x_h1LRzKIxLzVHnZW20P7pQqVp4t3__wXwfUA...,wd_OLr4cZHGaMiSfUlkoOIpldsIOmbqpK81TSZ_cvwG3xf...,Ketchup,4562,1678636043559,697
509,iGLGA4knP97PYNiytUzJwE-dosRYvWEgrRvtKUget6Y1mQ,yfYQr-dK69Bp2AbPP7HZqA-sBIE6kJLaVb4gSMizj5kHQFI,TAu1hwXk6_5fnyuR6PHLrcHDvni10n73A-9Kvp7coktUOa...,jimmy conwaay,4641,1678650256473,333
510,_gby9X8cluxiDvDC5_o7eHwo44sDfdC-R13PSv5JBNTspi...,8LD8WVBHySyzJXBUENQQVC8LKdYk0rb_0shhaQXRZFm5QI...,OVq1vMYRPpMuTcOO4IUMCfPWuhC_cMG-0eEXvbN84VeHR6...,Acquizhiwn,937,1678640416598,630


In [73]:
summoners = pd.read_csv('summoners.csv')
challengers = challengers.set_index('id')
summoners = summoners.set_index('id')

,platform,tier,id,summonerName
0,br1,challenger,qibcovz9vUQlqH30DqM354J6rUfas8DAvJknVfnKPaxm8pU,Brum basic
1,br1,challenger,nBtvfvSm-LAl6HvfN7yKSJGqVNzy_qfL7eBmqFWD-ICvrg,batman vermelho
2,br1,challenger,YRmiyU2YI2PK6xXdc4TlfUdypLlO4ScAMcSrw-iaRuwNzK...,oiiii
3,br1,challenger,l-QhwVfM7V9VtMV82y9f-RRnZlQ24CszAEWB-Ogi0mGL5w,Sleepy
4,br1,challenger,1tdg0nbHFRIErMoK77jaRnnSqIW2rYWYlHhB1Ggdbe5KRl...,sim sim


In [80]:
players = challengers.join(summoners, on='id')

In [105]:
matchId_buffer, buffer_id = [], 1
for platform, puuid in zip(players.platform, players.puuid):
    resp = None
    try:
        resp = lol_watcher.match.matchlist_by_puuid(
            platform, puuid, queue=420, 
            start_time=start_timestamp, end_time=end_timestamp)
    except ApiError as err:
        if err.response.status_code == 429:
            print('Retry-after:', err.response.headers['Retry-After'])
            time.sleep(err.response.headers['Retry-After'])
        elif err.response.status_code == 404:
            print('Acc not found:', puuid)
            continue
        else:
            print(err.response._content)
            continue
    if resp is not None:
        matchId_buffer += resp
    if len(matchId_buffer) > 500:
        with open(f'matchIds_{buffer_id}.json', 'w') as f:
            f.write(json.dumps(matchId_buffer))
        matchId_buffer, buffer_id = [], buffer_id + 1
        
with open(f'matchIds_{buffer_id}.json', 'w') as f:
            f.write(json.dumps(matchId_buffer))

b'{"status":{"message":"Bad request - Invalid routing value ph2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value tw2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value vn2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value ph2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value tw2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value vn2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value ph2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value tw2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value vn2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value ph2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing value tw2","status_code":400}}'
b'{"status":{"message":"Bad request - Invalid routing 

In [107]:
# with open('matchIds_1.json', 'r') as f:
#     lst = json.loads(f.read())
#     print(lst)

In [95]:
get_platform = lambda match_id: match_id.split('_')[0].lower()
get_platform(matchId_buffer[0])

'br1'

In [ ]:
matchId_buffer, bufferId = [], 1
processed = set()

In [124]:
for f in cwd.glob('matchIds_*.json'):
    if f.name in processed: continue
    
    print(f.name)
    processed.add(f.name)
    
    matchId_buffer = []
    with f.open() as file:
        matchId_buffer = json.loads(file.read())
    print(len(matchId_buffer))
    
    participants_buffer = []
    matches_buffer = []
    for match_id in matchId_buffer:
        platform = get_platform(match_id)
        resp = None
        try:
            resp = lol_watcher.match.by_id(platform, match_id)
        except ApiError as err:
            if err.response.status_code == 429:
                # print('Retry-after:', err.response.headers['Retry-After'])
                time.sleep(1)
            elif err.response.status_code == 404:
                print('Acc not found:', puuid)
                continue
            else:
                print(err.response._content)
                continue
            
        participants = []
        teams = {'matchId': match_id, 'platform': platform, 'teams': resp['info']['teams']}
        
        for p in resp['info']['participants']:
            info = {'matchId': match_id, 'platform': platform}
            for k in schema['participant_keys']:
                info[k] = p[k]
            participants_buffer.append(info)  
        matches_buffer.append(teams)
        
    with open(f'match_participant_{bufferId}.json', 'w') as f:
        f.write(json.dumps(participants_buffer))
    
    with open(f'match_team_{bufferId}.json', 'w') as f:
        f.write(json.dumps(matches_buffer))
    bufferId += 1
    

matchIds_45.json
503
b'{"status":{"message":"Service unavailable","status_code":503}}'
matchIds_12.json
506
matchIds_53.json
507
matchIds_24.json
510
matchIds_5.json
513
matchIds_32.json
504
matchIds_49.json
510


In [ ]:
processed = set()
for f in cwd.glob('match_team_*.json'):
    if f.name in processed: continue

In [3]:
from pathlib import Path
cwd = Path('.')

import pandas as pd
import json
matchIds = set()
dfs = []
for f in cwd.glob('match_participant_*.json'):
    dfs.append(pd.read_json(f.name))

In [4]:
df = pd.concat(dfs)
df.shape

(274210, 19)

In [ ]:
df.reset_index(drop=True)
df = df.groupby(['matchId', 'puuid']).first()

In [15]:
df.to_csv('participant_records.csv', index=True)

In [9]:
import pandas as pd
df = pd.read_csv('participant_records.csv')
df['position'] = df[['lane', 'role']].apply(lambda x: x.lane if x.lane in ['TOP', 'MIDDLE', 'JUNGLE'] else 'SUPPORT' if x.role == 'SUPPORT' else 'BOTTOM', axis=1)

In [14]:
df.to_csv('participant_records.csv', index=False)

In [16]:
eune = "Colombia, Costa Rica, Ecuador, El Salvador, Guatemala, Honduras, Mexico, Nicaragua, Panama, Paraguay, Peru"
eune.split(', ')

['Colombia',
 'Costa Rica',
 'Ecuador',
 'El Salvador',
 'Guatemala',
 'Honduras',
 'Mexico',
 'Nicaragua',
 'Panama',
 'Paraguay',
 'Peru']